In [32]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpa

In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-AWS-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [34]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [35]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vote_count = vine_df.filter("total_votes >= 20")
vote_count.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [36]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
filter_votes = vote_count.filter((vote_count["helpful_votes"]/vote_count["total_votes"])>=0.5)
filter_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [37]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_filter = filter_votes.filter(filter_votes.vine == "Y")
paid_filter.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|     

In [38]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_filter = filter_votes.filter(filter_votes.vine == "N")
unpaid_filter.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [39]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
# Total number of reviews unfiltered
total_votes = vine_df.count()
total_votes

1785997

In [40]:
# 5 star reviews from filtered votes
five_star = filter_votes.filter(filter_votes.star_rating == 5).count()
five_star

15711

In [41]:
# Total paid votes
total_paid_votes_df = paid_filter.count()
total_paid_votes_df

94

In [42]:
# PAID 5 star
five_star_paid = paid_filter.filter(paid_filter.star_rating == 5).count()
five_star_paid

48

In [43]:
# Percentage 5 star paid
fsp_percent = (five_star_paid/total_paid_votes_df) * 100
fsp_percent

51.06382978723404

In [44]:
# Total votes unpaid
total_unpaid_votes_df = unpaid_filter.count()
total_unpaid_votes_df

40471

In [45]:
# UNPAID 5 star
five_star_unpaid = unpaid_filter.filter(unpaid_filter.star_rating == 5).count()
five_star_unpaid

15663

In [46]:
fsu_percent = (five_star_unpaid/total_unpaid_votes_df) * 100
fsu_percent

38.701786464381904

In [47]:
# Total vine members
paid_filter_total = vine_df.filter(vine_df.vine == "Y")
paid_filter_total.show()
paid_filter_total.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RX3CK2RLSAF7T|          5|            0|          0|   Y|                N|
|R253CC14DRG5WQ|          3|            0|          0|   Y|                N|
| R1ZXWP3JKVXMU|          5|            0|          0|   Y|                N|
|R1I01YYQ35M80G|          5|            0|          0|   Y|                N|
| RWKQQPG100CQ4|          4|            2|          4|   Y|                N|
|R1R0J8S1VGALUY|          1|            1|          2|   Y|                N|
|R28165EG0N9CSN|          4|            1|          2|   Y|                N|
|R1CH0ZXGIMVEFC|          5|            1|          2|   Y|                N|
|R2BFEJHEKH4YWZ|          5|            3|          4|   Y|                N|
| R73CGNRBSWLLM|          5|            0|          0|   Y|     

4291

In [48]:
# Paid verified purchase with filter
verified_paid = paid_filter.filter(paid_filter.verified_purchase == "Y")
verified_paid.show()
verified_paid.count()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



0

In [49]:
# 5 star verified
verified_paid_five = verified_paid.filter(verified_paid.star_rating == 5)
verified_paid_five.show()
verified_paid_five.count()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



0

In [50]:
# Total non vine members
unpaid_filter_total = vine_df.filter(vine_df.vine == "N")
unpaid_filter_total.show()
unpaid_filter_total.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

1781706

In [51]:
# Unpaid verified purchase
verified_unpaid = unpaid_filter.filter(unpaid_filter.verified_purchase == "Y")
verified_unpaid.show()
verified_unpaid.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|                Y|
|R16621F16PQN5A|          3|           24|         31|   N|                Y|
| RNEMGSUKJ4535|          2|           21|         25|   N|                Y|
|R3R2UKBFKP1IQ4|          1|           19|         23|   N|                Y|
|R2T90XM5G2DBZA|          3|           15|         22|   N|                Y|
|R12E2W0XOV3I2S|          1|           24|         30|   N|     

10106

In [52]:
# Unpaid 5 star verified
verified_unpaid_five = verified_unpaid.filter(verified_unpaid.star_rating == 5)
verified_unpaid_five.show()
verified_unpaid_five.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDX2ZZ46AM343|          5|           32|         34|   N|                Y|
| R60GI4Z1CNGGV|          5|           17|         20|   N|                Y|
|R2FAARI3JQO9XQ|          5|           29|         30|   N|                Y|
|R3M42FPE0VJA0U|          5|           20|         22|   N|                Y|
| RCK4WXLGD3HDY|          5|           72|         81|   N|                Y|
|R18YE5XIUB40P9|          5|           18|         20|   N|                Y|
|R2WPAHM90BCWUN|          5|          210|        227|   N|                Y|
|R2CUQ7BSCBOXVB|          5|           24|         25|   N|                Y|
| RZ96PWGUW68WP|          5|           33|         37|   N|                Y|
|R3RY2C3PZMO9S9|          5|           23|         23|   N|     

3862

In [53]:
# Percentage verified purchase
fsuv_percent = (verified_unpaid_five.count()/verified_unpaid.count()) * 100
fsuv_percent

38.21492182861667

In [54]:
# To be sure double filter
ver_paid = paid_filter.filter(paid_filter.verified_purchase == "Y").filter(paid_filter.star_rating == 5).count()
ver_paid

0

In [55]:
ver_unpaid = unpaid_filter.filter(unpaid_filter.verified_purchase == "Y").filter(unpaid_filter.star_rating == 5).count()
ver_unpaid

3862